In [22]:
import json
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from phd_project.config.loader import load_config

cfg = load_config() 

In [23]:
DATA_ROOT = Path(r"C:\Users\clemettn\Documents\phd\data"
                 r"\eshm20_subduction_vrancea_datasets")

inslab_db_fp = DATA_ROOT / "hellenic_inslab_db.json"
interface_db_fp = DATA_ROOT / "hellenic_interface_db.json"
vrancea_db_fp = DATA_ROOT / "vrancea_db.json"

flatfile_folder = Path(r"C:\Users\clemettn\Documents\phd\data\gm_flatfiles")
esm_flatfile_fp = flatfile_folder / "ESM_flatfile_SA.csv"

site_model_fp = Path(cfg["data"]["eshm20_site_model"])

In [24]:
# load the files
with open(vrancea_db_fp, "r") as file:
    vrancea_db = json.load(file)

with open(inslab_db_fp, "r") as file:
    inslab_db = json.load(file)

with open(interface_db_fp, "r") as file:
    interface_db = json.load(file)

# load esm flatfile
# load all columns as strings to prevent any unintended formatting changes
# e.g. station code 0703 being loaded as int 703...
# the numerical columns can be converted to floats ad hoc
esm_db = pd.read_csv(esm_flatfile_fp, sep=";", 
                     dtype=str)

In [25]:
# calculate the hypocentral distance for every record
esm_db["epi_dist"] = pd.to_numeric(esm_db["epi_dist"])
esm_db["ev_depth_km"] = pd.to_numeric(esm_db["ev_depth_km"])
hypo_dist = np.sqrt(esm_db["epi_dist"] ** 2 + esm_db["ev_depth_km"] ** 2)

idx = np.where(esm_db.columns == "epi_dist")[0][0]
esm_db.insert(loc=idx+1, column="hypo_dist", value=hypo_dist)

# some data conversions:
esm_db["EMEC_Mw"] = pd.to_numeric(esm_db["EMEC_Mw"])

In [26]:
# checking that all of the records in the json files (subsets) are in the
# esm databank
nv_in_esm = []
nv_not_in_esm = []
nss_in_esm = []
nss_not_in_esm = []
nsi_in_esm = []
nsi_not_in_esm = []

# vrancea
for ii, record in enumerate(vrancea_db["records"]):

    # the metadata required to unambiquously identify each recording
    network_code = record["site"]["network_code"]
    station_code = record["site"]["code"]
    event_id = record["event"]["id"]

    df = esm_db[(esm_db["network_code"] == network_code) &
                (esm_db["station_code"] == station_code) &
                (esm_db["event_id"] == event_id)]
    
    if len(df.index) == 1:
        nv_in_esm.append(df.index[0])
    else:
        nv_not_in_esm.append(ii)

# subduction slab
for ii, record in enumerate(inslab_db["records"]):

    # the metadata required to unambiquously identify each recording
    network_code = record["site"]["network_code"]
    station_code = record["site"]["code"]
    event_id = record["event"]["id"]

    df = esm_db[(esm_db["network_code"] == network_code) &
                (esm_db["station_code"] == station_code) &
                (esm_db["event_id"] == event_id)]
    
    if len(df.index) == 1:
        nss_in_esm.append(df.index[0])
    else:
        nss_not_in_esm.append(ii)

# subduction interface
for ii, record in enumerate(interface_db["records"]):

    # the metadata required to unambiquously identify each recording
    network_code = record["site"]["network_code"]
    station_code = record["site"]["code"]
    event_id = record["event"]["id"]

    df = esm_db[(esm_db["network_code"] == network_code) &
                (esm_db["station_code"] == station_code) &
                (esm_db["event_id"] == event_id)]
    
    if len(df.index) == 1:
        nsi_in_esm.append(df.index[0])
    else:
        nsi_not_in_esm.append(ii)

print(f"Number of records from the Vrancea subset not in the ESM flatfile: "
      f"{len(nv_not_in_esm)}")
print(f"Number of records from the Sub. Inslab subset not in the ESM flatfile: "
      f"{len(nss_not_in_esm)}")
print(f"Number of records from the Sub. Interface subset not in the ESM flatfile: "
      f"{len(nsi_not_in_esm)}")

Number of records from the Vrancea subset not in the ESM flatfile: 0
Number of records from the Sub. Inslab subset not in the ESM flatfile: 0
Number of records from the Sub. Interface subset not in the ESM flatfile: 0


In [27]:
esm_db.head()

,event_id,event_time,ISC_ev_id,USGS_ev_id,INGV_ev_id,EMSC_ev_id,ev_nation_code,ev_latitude,ev_longitude,ev_depth_km,...,rotD00_T3_000,rotD00_T3_500,rotD00_T4_000,rotD00_T4_500,rotD00_T5_000,rotD00_T6_000,rotD00_T7_000,rotD00_T8_000,rotD00_T9_000,rotD00_T10_000
0,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.012796,0.010137,0.008776,0.006618,0.003835,0.002287,0.001832,0.001362,0.001165,0.00083
1,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.004239,0.002791,0.002294,0.001816,0.001211,0.00101,0.000785,0.000627,0.000455,0.000352
2,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.003386,0.00275,0.002354,0.001539,0.001046,0.00077,0.000427,0.000407,0.000343,0.000246
3,AL-2016-0001,2016-09-12 17:53:52,NaN,us10006p3d,NaN,NaN,MK,41.993,21.4464,16.23,...,0.01176,0.008782,0.005104,0.003737,0.00265,0.001621,0.001232,0.000986,0.000794,0.000685
4,AL-2016-0001,2016-09-12 17:53:52,NaN,us10006p3d,NaN,NaN,MK,41.993,21.4464,16.23,...,0.001531,0.000948,0.000892,0.000589,0.000447,0.000287,0.000237,0.000178,0.000154,0.000134


In [39]:
# add some metadata columns to the esm_db
esm_db_mod = esm_db.copy()
# xvf: the distance to the volcanic front in km
# approximated for each recording using  the neareast site from the ESHM20 site
# model

# load the site model 
site_model = pd.read_csv(site_model_fp, sep=",")
site_model = gpd.GeoDataFrame(site_model, 
                            geometry=gpd.points_from_xy(
                               site_model.lon, site_model.lat),
                            crs="EPSG:4326")

# match each esm_db site to the nearest site in the site model
esm_gdf = gpd.GeoDataFrame(esm_db, 
                            geometry=gpd.points_from_xy(
                                esm_db.st_longitude, esm_db.st_latitude),
                            crs="EPSG:4326")

nearest_sites = site_model.sindex.nearest(esm_gdf.geometry, return_all=False)
xvf_values = site_model.iloc[nearest_sites[1]]["xvf"].values
z1pt0_values = site_model.iloc[nearest_sites[1]]["z1pt0"].values
region_values = site_model.iloc[nearest_sites[1]]["region"].values

n_added_columns = 11
# add columns needed for the openquake context array
# magnitude "mag": where possible used the EMEC_Mw, otherwise fall back on the
# default Mw
esm_db_mod["mag"] = esm_db_mod["EMEC_Mw"].combine_first(esm_db_mod["Mw"])
esm_db_mod["hypo_depth"] = esm_db_mod["ev_depth_km"]
# rake: where possible use the "es_rake" otherwise set to -90 (reverse faulting)
esm_db_mod["rake"] = esm_db_mod["es_rake"].fillna(-90)

esm_db_mod["rhypo"] = esm_db_mod["hypo_dist"]
# distance from rupture plane "rrup": where possible use rup_dist, otherwise
# fall back on hypo_dist
esm_db_mod["rrup"] = esm_db_mod["rup_dist"].combine_first(esm_db_mod["hypo_dist"])
esm_db_mod["rjb"] = esm_db_mod["JB_dist"].combine_first(esm_db_mod["epi_dist"])
esm_db_mod["xvf"] = xvf_values


esm_db_mod["vs30"] = esm_db_mod["vs30_m_sec"].combine_first(esm_db_mod["vs30_m_sec_WA"])
# differentiate between vs30 from measurements or from slope
esm_db_mod["vs30measured"] = esm_db_mod["vs30_m_sec"].notna()
esm_db_mod["z1pt0"] = z1pt0_values
esm_db_mod["region"] = region_values

# reorder the dataframe columns for easier viewing
cols = esm_db_mod.columns.tolist()
cols_before = cols[:72]
cols_after = cols[72:-n_added_columns]
cols_to_move = cols[-n_added_columns:]
new_cols = cols_before + cols_to_move + cols_after
esm_db_mod = esm_db_mod[new_cols]
esm_db_mod.head()

,event_id,event_time,ISC_ev_id,USGS_ev_id,INGV_ev_id,EMSC_ev_id,ev_nation_code,ev_latitude,ev_longitude,ev_depth_km,...,rotD00_T3_000,rotD00_T3_500,rotD00_T4_000,rotD00_T4_500,rotD00_T5_000,rotD00_T6_000,rotD00_T7_000,rotD00_T8_000,rotD00_T9_000,rotD00_T10_000
0,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.012796,0.010137,0.008776,0.006618,0.003835,0.002287,0.001832,0.001362,0.001165,0.00083
1,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.004239,0.002791,0.002294,0.001816,0.001211,0.00101,0.000785,0.000627,0.000455,0.000352
2,AL-2014-0005,2014-01-20 14:40:48,NaN,usb000m4t2,NaN,NaN,AL,41.2985,19.4484,28.26,...,0.003386,0.00275,0.002354,0.001539,0.001046,0.00077,0.000427,0.000407,0.000343,0.000246
3,AL-2016-0001,2016-09-12 17:53:52,NaN,us10006p3d,NaN,NaN,MK,41.993,21.4464,16.23,...,0.01176,0.008782,0.005104,0.003737,0.00265,0.001621,0.001232,0.000986,0.000794,0.000685
4,AL-2016-0001,2016-09-12 17:53:52,NaN,us10006p3d,NaN,NaN,MK,41.993,21.4464,16.23,...,0.001531,0.000948,0.000892,0.000589,0.000447,0.000287,0.000237,0.000178,0.000154,0.000134


In [40]:
# dataframes for each of the subsets
df_vr = esm_db_mod.loc[nv_in_esm]
df_ss = esm_db_mod.loc[nss_in_esm]
df_si = esm_db_mod.loc[nsi_in_esm]

In [41]:
# save the subsets as their own flatfiles
vr_fp = flatfile_folder / "vrancea_esm_flatfile.csv"
si_fp = flatfile_folder / "subduction_interface_esm_flatfile.csv"
ss_fp = flatfile_folder / "subduction_inslab_esm_flatfile.csv"

for df, fp in zip([df_vr, df_si, df_ss],
                  [vr_fp, si_fp, ss_fp]):
    df.to_csv(fp, sep=";")

    print(fp.name)
    print("No. Events: ", len(df.groupby("event_id")))
    print("No. Records: ", len(df))
    print("No. Stations: ", len(df["station_code"].unique()))
    print("Mw (min/max): ", 
        df["EMEC_Mw"].min(), df["EMEC_Mw"].max(), "\n")

vrancea_esm_flatfile.csv
No. Events:  64
No. Records:  938
No. Stations:  92
Mw (min/max):  3.8 7.5 

subduction_interface_esm_flatfile.csv
No. Events:  164
No. Records:  681
No. Stations:  167
Mw (min/max):  1.69 6.5 

subduction_inslab_esm_flatfile.csv
No. Events:  121
No. Records:  704
No. Stations:  178
Mw (min/max):  1.69 6.2 



In [42]:
# get the active shallow crust flatfile based on the filter criteria described 
# in section 4.2.1 of the ESHM20 report. Danciu et al. (2021)
# i) events must be classified as non-subduction (according to the 
# classification scheme indicated in the subsequent section), 
# ii) reported hypocentral depth must be shallower than 40 km, 
# iii) events must have ≥ 3 records, 
# iv) only records with high-pass filter frequency 𝑓Hp ≤ 0.8 𝑇 are retained
# for each period in the regression. 
# 
# According to the report the final database yields 18,222 records from 927 
# events (3.1 ≤ 𝑀w ≤ 7.4) recorded at 1829 stations (0 ≤ 𝑅JB(𝑘𝑚) ≤ 545).
deep_events = nv_in_esm + nsi_in_esm + nss_in_esm
asc_esm_df = esm_db_mod.loc[~esm_db.index.isin(deep_events)]    # (i)
asc_esm_df = asc_esm_df[asc_esm_df["ev_depth_km"] <= 40]    # (ii)
asc_esm_df = asc_esm_df.groupby("event_id").filter(lambda g: len(g) >= 3) # (iii)

# (iv)
# T = 8 # seconds e.g.
# asc_esm_df = asc_esm_df[(pd.to_numeric(asc_esm_df["U_hp"]) <= 0.8/T) &
#                         (pd.to_numeric(asc_esm_df["V_hp"]) <= 0.8/T)]

print("No. Events: ", len(asc_esm_df.groupby("event_id")))
print("No. Records: ", len(asc_esm_df))
print("No. Stations: ", len(asc_esm_df["station_code"].unique()))
print("Mw (min/max): ", 
      asc_esm_df["EMEC_Mw"].min(), asc_esm_df["EMEC_Mw"].max())

# save the asc flatfile
asc_esm_df.to_csv(flatfile_folder / "active_shallow_crust_esm_flatfile.csv",
                  sep=";")

No. Events:  1068
No. Records:  19438
No. Stations:  1916
Mw (min/max):  3.05 7.4
